## Start

In [ ]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Partner

### Links: 
___

In [ ]:
# creating a table from s3

query_text = """--sql
    CREATE TABLE db1.no_video_on_stream_mobile_st_asgard_mv
    (
        `partner_uuid` String,
        `business_partner_uuid`  String,
        `updated_at` DateTime,
        `partner_uk_email` String,
        `report_date` Date
    )
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/uk_st_partner/year=*/month=*/*.csv','CSVWithNames')
    """

ch.query_run(query_text)

In [ ]:
# creating a table for materialized view

query_text = """--sql
    CREATE TABLE db1.uk_st_partner_ch
    (
        `partner_uuid` String,
        `business_partner_uuid` String,
        `updated_at` DateTime,
        `partner_uk_email` String,
        `report_date` Date
    )
    ENGINE = MergeTree()
    ORDER BY partner_uuid
    """

ch.query_run(query_text)

In [ ]:
# creating a materialized view

query_text = """--sql
    CREATE MATERIALIZED VIEW db1.no_video_on_stream_mobile_st_asgard_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.no_video_on_stream_mobile_st_asgard_ch AS
    SELECT
        *
    FROM db1.uk_st_partner
    """

ch.query_run(query_text)

___
## Tools
___
### query

In [ ]:
query_text = """--sql
    SELECT
        *
    FROM db1.no_video_on_stream_mobile_st_asgard_ch
    LIMIT 2
    """

ch.query_run(query_text)

### Drop ch

In [ ]:
query_text = """
    DROP TABLE db1.no_video_on_stream_mobile_st_asgard_ch
    """
ch.query_run(query_text)

### Drop mv

In [ ]:
query_text = """
    DROP TABLE db1.no_video_on_stream_mobile_st_asgard_mv
    """
ch.query_run(query_text)